In [2]:
# pip install opencv-python

import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd

In [1]:
def get_table(sims):
    """
    This function takes the output produced by either the compute_similarities \ 
    or compute_similarities_testsets function, and returns a pandas dataframe/table \
    and also saves it in excel.
    """
    
    data = {}
    rows = []

    for key, value in sims.items():
        if key[0] not in data:
            data[key[0]] = []
        if key[1] not in rows:
            rows.append(key[1])
        data[key[0]].append(value)
        
    data = {key[:key.rfind(".")]:value for key, value in data.items()}
    rows = [row[:row.rfind(".")] for row in rows]
        
    df = pd.DataFrame(data, index=rows)
    #df.to_excel('output.xlsx')
    return df.T
    
def get_table1(sims):
    """
    This function takes the output produced by either the compute_similarities 
    or compute_similarities_testsets function, and returns a pandas dataframe/table 
    and also saves it in excel.
    """
    
    data = {}
    rows = []
    
    mapping = {
        "kast_nk.jpg": "closet_nk.jpg",
        "speeltafel_nk.png": "card_table_nk.png",
        "tafel_nk.jpg": "table_nk.jpg",
        "dressoir_nk.jpg": "dresser_nk.jpg",
        "stoel_nk.jpg": "chair_nk.jpg",
        "kast_mccp.jpg": "closet_mccp.jpg",
        "speeltafel_mccp.png": "card_table_mccp.png",
        "tafel_mccp.jpg": "table_mccp.jpg",
        "dressoir_mccp.jpg": "dresser_mccp.jpg",
        "stoel_mccp.jpg": "chair_mccp.jpg",
    }

    for (key1, key2), value in sims.items():
        file1 = os.path.basename(key1)
        file2 = os.path.basename(key2)
        
        if mapping[file1] not in data:
            data[mapping[file1]] = []
        if mapping[file2] not in rows:
            rows.append(mapping[file2])
        
        value = np.round(value, 3)
        data[mapping[file1]].append(value)
        
    data = {key[:key.rfind(".")]: value for key, value in sorted(data.items())}
    rows = [row[:row.rfind(".")] for row in rows]
    rows_indices_begin = list(range(len(rows)))
    data_values = list(data.values())
    rows_indices_end = sorted(rows_indices_begin, key=lambda i: rows[i])
    rows = sorted(rows)
    
    for key, value in data.items():
        new_value = []
        for i in rows_indices_end:
            new_value.append(value[i])
            
        data[key] = new_value
    
    df = pd.DataFrame(data, index=rows)
    #df.to_excel('output.xlsx')
    return df.T

<>:2: SyntaxWarning: invalid escape sequence '\ '
<>:2: SyntaxWarning: invalid escape sequence '\ '
C:\Users\used\AppData\Local\Temp\ipykernel_10624\1145853711.py:2: SyntaxWarning: invalid escape sequence '\ '
  """


In [4]:
def extract_orb_features(image_path):
    # Load image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"Image at path {image_path} could not be loaded.")
    #, scaleFactor=scaleFactor, nlevels=nlevels
    # Initialize ORB detector
    # orb = cv2.ORB_create(nfeatures=500, scaleFactor=1.2, nlevels=8, WTA_K=2, 
    #                      scoreType=cv2.ORB_HARRIS_SCORE, patchSize=31, fastThreshold=20)
    #Best parameters: (100, 1.1, 12, 4, 0, 21, 10)
    orb = cv2.ORB_create(nfeatures=200,scaleFactor=1.1,nlevels=8,WTA_K=3, 
                          scoreType=cv2.ORB_HARRIS_SCORE, patchSize=41, fastThreshold=30)
    
    # Find keypoints and descriptors
    kp, des = orb.detectAndCompute(img, None)
    return kp, des


# def match_descriptors(des1, des2):
#     matches = []
#     for i, d1 in enumerate(des1):
#         distances = np.linalg.norm(des2 - d1, axis=1)
#         min_idx = np.argmin(distances)
#         match = cv2.DMatch(_queryIdx=i, _trainIdx=min_idx, _distance=distances[min_idx])
#         matches.append(match)
#     return matches

def match_descriptors(des1, des2):
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)
    return matches

def ratio_test(matches, ratio_threshold=0.75):
    good_matches = []
    for i, (query_idx, train_idx, distance) in enumerate(matches):
        if i < len(matches) - 1:
            next_distance = matches[i + 1][2]
            if distance < ratio_threshold * next_distance:
                good_matches.append((query_idx, train_idx))
    return good_matches

def compute_orb_similarity(image_path1, image_path2):
    # Extract ORB features
    kp1, des1 = extract_orb_features(image_path1)
    kp2, des2 = extract_orb_features(image_path2)
    
    if des1 is None or des2 is None:
        return 0.0

    # Match descriptors manually
    matches = match_descriptors(des1, des2)
    
    # Apply ratio test
    # good_matches = ratio_test(matches)
    
    # if len(good_matches) < 4:
    #     return 0.0
    
    # src_pts = np.float32([kp1[i].pt for i, _ in good_matches]).reshape(-1, 1, 2)
    # dst_pts = np.float32([kp2[i].pt for _, i in good_matches]).reshape(-1, 1, 2)
    if len(matches) < 4:
        return 0.0
    
    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    
    # Use RANSAC to find the best transformation matrix
    H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 15)
    matches_mask = mask.ravel().tolist()

    num_inliers = np.sum(matches_mask)
    num_matches = len(matches)
    
    similarity_percentage = (num_inliers / num_matches) * 100
    return similarity_percentage


def compute_similarities_testsets(munich_testset, nk_testset, 
                                  munich_path="munich_testset", 
                                  nk_path="nk_testset"):
    """
    This function takes four arguments: 
    - munich_testset, which contains grayscaled images from the Munich database.
    - nk_testset, which contains grayscaled images from the NK collection API.
    - munich path, the path to the directory of the Munich images. 
    - nk_path, the path to the directory of the NK images. 
    
    It then computes the similarity using ORB features.
    It then saves the similarity and the two images as key-value pairs in a dictionary.
    """
    
    similarities = {}
    for nk_img in nk_testset:
        nk_img_path = os.path.join(nk_path, nk_img)
        for munich_img in munich_testset:
            munich_img_path = os.path.join(munich_path, munich_img)
            try:
                similarity = compute_orb_similarity(nk_img_path, munich_img_path)
                similarities[(nk_img, munich_img)] = similarity
            except ValueError as e:
                print(e)
        
    return similarities

# Example usage:
nk_path = "nk_testset"
munich_path = "munich_testset"

if not os.path.exists(nk_path):
    raise FileNotFoundError(f"The path {nk_path} does not exist.")
if not os.path.exists(munich_path):
    raise FileNotFoundError(f"The path {munich_path} does not exist.")

nk_testset = os.listdir(nk_path)
munich_testset = os.listdir(munich_path)

sims = compute_similarities_testsets(munich_testset, nk_testset, munich_path, nk_path)

get_table1(sims)

,card_table_mccp,chair_mccp,closet_mccp,dresser_mccp,table_mccp
card_table_nk,37.037,35.714,34.783,31.250,36.364
chair_nk,26.667,50.000,33.333,33.333,35.294
closet_nk,33.333,43.750,33.333,42.857,36.000
dresser_nk,35.484,41.935,37.931,57.143,31.579
table_nk,23.077,34.783,31.034,42.857,26.923


In [11]:
image_folder = 'test_dataset_no_back'

# List to hold the images
images_nk = []
images_mccp = []

# Loop through the files in the directory
for filename in os.listdir(image_folder):
    file_path = os.path.join(image_folder, filename)
    

    if 'nk' in filename:
        images_nk.append((filename, file_path))
    if 'mccp' in filename:
        images_mccp.append((filename, file_path))

In [13]:
import itertools

def test_orb_parameters(images_nk, images_mccp):
    nfeatures_list = [100, 200, 500, 800, 1000]
    scaleFactor_list = [1.1, 1.2, 1.3]
    nlevels_list = [4, 8, 12]
    WTA_K_list = [2, 3, 4]
    scoreType_list = [cv2.ORB_HARRIS_SCORE, cv2.ORB_FAST_SCORE]
    patchSize_list = [21, 31, 41]
    fastThreshold_list = [10, 20, 30]
    
    best_similarity = 0
    best_params = None
    
    for nfeatures, scaleFactor, nlevels, WTA_K, scoreType, patchSize, fastThreshold in itertools.product(
            nfeatures_list, scaleFactor_list, nlevels_list, WTA_K_list, scoreType_list, patchSize_list, fastThreshold_list):
        
        orb = cv2.ORB_create(nfeatures=nfeatures, scaleFactor=scaleFactor, nlevels=nlevels, WTA_K=WTA_K, 
                             scoreType=scoreType, patchSize=patchSize, fastThreshold=fastThreshold)
        similarity_score = 0
        for filename_nk, image_path1 in images_nk:
            for filename_mccp, image_path2 in images_mccp:
                same_item = False
                if filename_nk[:2].lower() == filename_mccp[:2].lower():
                    same_item = True
                kp1, des1 = extract_orb_features(image_path1, orb)
                kp2, des2 = extract_orb_features(image_path2, orb)
                
                if des1 is None or des2 is None:
                    continue
                
                matches = match_descriptors(des1, des2)
                if len(matches) < 4:
                    continue
                
                src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
                dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
                
                H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 15)
                if mask is None:
                    continue
                
                matches_mask = mask.ravel().tolist()
                num_inliers = np.sum(matches_mask)
                num_matches = len(matches)
                if same_item:
                    similarity_score += (num_inliers / num_matches) * 100 * 4
                else:
                    similarity_score -= (num_inliers / num_matches) * 100
        
        if similarity_score > best_similarity:
            best_similarity = similarity_score
            best_params = (nfeatures, scaleFactor, nlevels, WTA_K, scoreType, patchSize, fastThreshold)
    
    return best_similarity, best_params

def extract_orb_features(image_path, orb):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"Image at path {image_path} could not be loaded.")
    
    kp, des = orb.detectAndCompute(img, None)
    return kp, des

def match_descriptors(des1, des2):
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)
    return matches

# Example usage:
best_similarity, best_params = test_orb_parameters(images_nk, images_mccp)
print("Best similarity:", best_similarity)
print("Best parameters:", best_params)


Best similarity: 155.4596694326829
Best parameters: (200, 1.1, 8, 3, 0, 41, 30)


In [ ]:
import matplotlib.pyplot as plt

def plot_distances(matches):
    distances = [match.distance for match in matches]
    plt.plot(range(len(distances)), distances, marker='o', linestyle='-')
    plt.xlabel('Match Index')
    plt.ylabel('Distance')
    plt.title('Distances of Matches')
    plt.grid(True)
    plt.show()

# Assuming matches is a list of cv2.DMatch objects sorted by distance
sorted_matches = sorted(matches, key=lambda x: x.distance)

# Plot the distances to analyze the trend
plot_distances(sorted_matches)